In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [ ]:
from requests_html import HTMLSession
session = HTMLSession()

In [ ]:
base_site = 'https://www.avito.ru/'
a = 'https://www.avito.ru/kazan/avtomobili/volkswagen-ASgBAgICAUTgtg24mSg?cd=1&radius=0&p='
parsing_links = [(a + str(i)) for i in range(1,15)]

In [ ]:
test = session.get(parsing_links[1])

In [ ]:
test.status_code

In [ ]:
with open('avito_test.html', 'wb') as file:
    file.write(test.content)

In [ ]:
all_links = []

for i in parsing_links:
    r = session.get(i)
    snippet_links = r.html.find('.snippet-link')
    stop_elements = '?'
    links = [snippet_links.attrs['href'] for snippet_links in snippet_links if stop_elements not in snippet_links.attrs['href']]    
    full_urls = [urljoin(base_site, url) for url in links]
    all_links.extend(full_urls)

In [ ]:
len(all_links)

In [ ]:
opts = Options()
opts.headless = True
browser = webdriver.Chrome(options=opts, executable_path=r'/Users/dmitrii/Documents/ChromeWebDriver/chromedriver')

In [ ]:
marka_car = []
model_car = []
year_car = []
probeg_car = []
condition_car = []
owners_car = []
price_car = []

In [ ]:
wrong_urls = []

In [ ]:
for i in range(400,751):
    try:
        browser.get(all_links[i])
        car = browser.page_source
        soup = BeautifulSoup(car, 'html5lib')
    
        item_params = soup.find_all(class_ = "item-params-list-item")
    
        marka = str(item_params[0].text.strip(' Марка:'))
        marka_car.append(marka)
    
        model = str(item_params[1].text.strip(' Модель:'))
        model_car.append(model)
    
        year = str(item_params[4].text.strip('Год выпуска:'))
        year_car.append(year)
    
        probeg = str(item_params[5].text.strip('Пробег: \xa0км'))
        probeg_car.append(probeg)
    
        condition = str(item_params[6].text.strip(' ').strip('Состояние:').strip(' '))
        condition_car.append(condition)
    
        owners = str(item_params[7].text.strip(' ').strip('Владельцев по ПТС:'))
        owners_car.append(owners)
    
        price = soup.find(class_ ='js-item-price')
        real_price = int(price.attrs['content'])
        price_car.append(real_price)
    
    except Exception:
        wrong_urls.append(all_links[i])
        continue

In [ ]:
avito_cars = pd.DataFrame()

In [ ]:
avito_cars["Марка"] = marka_car
avito_cars["Модель"] = model_car
avito_cars["Год выпуска"] = year_car
avito_cars["Пробег"] = probeg_car
avito_cars["Состояние"] = condition_car
avito_cars["Владельцев по ПТС"] = owners_car
avito_cars["Цена"] = price_car

In [ ]:
avito_cars

In [ ]:
avito_cars.to_csv('avito_cars.csv')

In [ ]:
session.close()

In [ ]:
browser.quit()